# Intro

This notebook shows an example how to use HexTractor to transform tabular data to heterogeneous graph.

# Load libs

In [1]:
import rootutils

root = rootutils.setup_root(".", dotenv=True, pythonpath=True, cwd=False)

In [2]:
import pandas as pd
import hextractor.data_sources as data_sources
import hextractor.structures as structures
import hextractor.extraction as hextract

# Single-table data case

We will start with the simples example - where all data is in a single table. The same entity (e.g. company) can be repeated mutliple times in the table - each row represents it relation with other entities e.g. company + employee. HexTractor will handle such duplication, extracting only unique entities and relations between them.


In [3]:
df = pd.DataFrame(
    [
        (1, 100, 1000, 0, 0, 25, 0, [1, 2, 3]),
        (1, 100, 1000, 1, 1, 35, 1, [1, 2]),
        (1, 100, 1000, 3, 3, 45, 0, [3, 4]),
        (2, 5000, 100000, 4, 1, 18, 1, [1, 4]),
        (2, 5000, 100000, 5, 1, 20, 1, [1, 1]),
        (2, 5000, 100000, 6, 4, 31, 0, [1, 2]),
    ],
    columns=[
        "company_id",
        "company_employees",
        "company_revenue",
        "employee_id",
        "employee_occupation",
        "employee_age",
        "employee_promotion",
        "tags",
    ],
)

df

,company_id,company_employees,company_revenue,employee_id,employee_occupation,employee_age,employee_promotion,tags
0,1,100,1000,0,0,25,0,"[1, 2, 3]"
1,1,100,1000,1,1,35,1,"[1, 2]"
2,1,100,1000,3,3,45,0,"[3, 4]"
3,2,5000,100000,4,1,18,1,"[1, 4]"
4,2,5000,100000,5,1,20,1,"[1, 1]"
5,2,5000,100000,6,4,31,0,"[1, 2]"


## Prepare graph specs

In [6]:
company_node_params = structures.NodeTypeParams(
    node_type_name="company",
    id_col="company_id",
    attributes=("company_employees", "company_revenue"),
    attr_type="float",
)

company_tags_node_params = structures.NodeTypeParams(
    node_type_name="tag",
    multivalue_source=True,
    id_col="tags",
)

employee_node_params = structures.NodeTypeParams(
    node_type_name="employee",
    id_col="employee_id",
    attributes=("employee_occupation", "employee_age"),
    target_col="employee_promotion",
    attr_type="long",
)

company_has_emp_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="employee",
    source_id_col="company_id",
    target_id_col="employee_id",
)

company_has_tag_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="tag",
    source_id_col="company_id",
    target_id_col="tags",
    multivalue_target=True
)

single_df_specs = data_sources.DataFrameSpecs(
    name="df1",
    node_params=(
        company_node_params,
        employee_node_params,
        company_tags_node_params,
    ),
    edge_params=(company_has_emp_edge_params, company_has_tag_edge_params),
    data_frame=df,
)

graph_specs = data_sources.GraphSpecs(data_sources=(single_df_specs,))

## Extract graph

In [7]:
hetero_g = hextract.extract_data(graph_specs)

/home/filip/projects/science/hextractor/hextractor/data_sources.py:147: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  id_counts = pd.value_counts(node_ids).max()
/home/filip/projects/science/hextractor/hextractor/data_sources.py:147: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  id_counts = pd.value_counts(node_ids).max()


In [8]:
hetero_g

HeteroData(
  company={ x=[3, 2] },
  employee={ x=[7, 2] },
  tag={ x=[5] },
  (company, has, employee)={ edge_index=[2, 6] },
  (company, has, tag)={ edge_index=[2, 7] }
)

# Multi-table data case

In this case we have multiple tables, each representing different entity type. We will show how to extract graph from such data. This is how the data is usually represented in a database or a normalized data warehouse.

In [9]:
df_company = pd.DataFrame(
    {
        "company_id": [1, 2],
        "company_employees": [100, 5000],
        "company_revenue": [1000, 100000],
    }
)

df_employee = pd.DataFrame(
    {
        "employee_id": [0, 1, 3, 4, 5, 6],
        "employee_occupation": [0, 1, 3, 1, 1, 4],
        "employee_age": [25, 35, 45, 18, 20, 31],
        "employee_promotion": [0, 1, 0, 1, 1, 0],
    }
)

df_tags = pd.DataFrame({
    'tag': [1, 2, 3, 4]
})

df_company_2_emplopyee = pd.DataFrame(
    {
        "company_id": [1, 1, 1, 2, 2, 2],
        "employee_id": [0, 1, 3, 4, 5, 6],
    }
)

df_company_2_tag = pd.DataFrame(
    {
        "company_id": [1, 1, 1, 2, 2, 2],
        "tags": [[1, 2, 3], [1, 2], [3, 4], [1, 4], [1, 1], [1, 2]],
    }
)

## Prepare graph specs

In [15]:
company_node_params = structures.NodeTypeParams(
    node_type_name="company",
    id_col="company_id",
    attributes=("company_employees", "company_revenue"),
    attr_type="float",
)
company_df_source = data_sources.DataFrameSpecs(
    name="df1", node_params=(company_node_params,), data_frame=df_company
)

company_tags_node_params = structures.NodeTypeParams(
    node_type_name="tag",
    multivalue_source=False,
    id_col="tag",
    id_as_attr=True,
)
tag_df_source = data_sources.DataFrameSpecs(
    name="df2", node_params=(company_tags_node_params,), data_frame=df_tags
)

employee_node_params = structures.NodeTypeParams(
    node_type_name="employee",
    id_col="employee_id",
    attributes=("employee_occupation", "employee_age"),
    label_col="employee_promotion",
    attr_type="long",
)
employee_df_source = data_sources.DataFrameSpecs(
    name="df3", node_params=(employee_node_params,), data_frame=df_employee
)

company_has_emp_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="employee",
    source_id_col="company_id",
    target_id_col="employee_id",
)
company_has_emp_edge_df_source = data_sources.DataFrameSpecs(
    name="df4",
    edge_params=(company_has_emp_edge_params,),
    data_frame=df_company_2_emplopyee,
)

company_has_tag_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="tag",
    source_id_col="company_id",
    target_id_col="tags",
    multivalue_target=True,
)
company_has_tag_edge_df_source = data_sources.DataFrameSpecs(
    name="df5",
    edge_params=(company_has_tag_edge_params,),
    data_frame=df_company_2_tag,
)

graph_specs_multisource = data_sources.GraphSpecs(
    data_sources=(
        company_df_source,
        employee_df_source,
        tag_df_source,
        company_has_emp_edge_df_source,
        company_has_tag_edge_df_source,
    )
)

### Extract graph

In [16]:
hetero_g_multi = hextract.extract_data(graph_specs_multisource)

/home/filip/projects/science/hextractor/hextractor/data_sources.py:147: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  id_counts = pd.value_counts(node_ids).max()
/home/filip/projects/science/hextractor/hextractor/data_sources.py:147: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  id_counts = pd.value_counts(node_ids).max()
/home/filip/projects/science/hextractor/hextractor/data_sources.py:147: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  id_counts = pd.value_counts(node_ids).max()


In [17]:
hetero_g_multi

HeteroData(
  company={ x=[3, 2] },
  employee={
    x=[7, 2],
    y=[7],
  },
  tag={ x=[5, 1] },
  (company, has, employee)={ edge_index=[2, 6] },
  (company, has, tag)={ edge_index=[2, 7] }
)

In [18]:
hetero_g_multi['company', 'has', 'tag'].edge_index

tensor([[1, 1, 1, 1, 2, 2, 2],
        [1, 2, 3, 4, 1, 2, 4]])